In [1]:
import os
import subprocess
import sys
import time
import matplotlib
matplotlib.use('Agg')
import librosa
import numpy as np
import pandas as pd
import pydub
import sklearn as sk
from numpy import *
from tqdm import *
import glob
%matplotlib inline
import matplotlib.pyplot as plt
from librosa import display
from sklearn.metrics import accuracy_score

In [2]:
X=[]
y=[]
z=[]

In [27]:
files = glob.glob("D:/iitd/SEM6/COP315/xeno_merged/"+"**", recursive=False)
i=0
for file in files:
    mp3s = glob.glob(file+"/**.npy", recursive=False)
    i+=1
    print(file[31:])
    for mp3 in mp3s:
        a=np.load(mp3);
        if(np.shape(a)==(170,1000)):
            x=a-mean(a)
            X.append(x)
            y.append(i)
            z.append(file[32:])
        else:
            print(np.shape(a))
        

\alexandrine_parakeet
\bulbul
\common_myna
\house_crow
\house_sparrow
\indian_peafowl
\indian_robin
\jungle_babbler
\koel
\rock_dove
\tailor_bird


In [28]:
bird_df=pd.DataFrame({'spec':X,'class no':y,'class name':z})

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [32]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(170,20), activation='relu', input_shape=(170,1000,1)))
model.add(MaxPooling2D(pool_size=(1,981)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(12, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 1, 981, 128)       435328    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 12)                780       
Total para

In [33]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [34]:
np.shape(X_train)

(1711, 170, 1000)

In [35]:
X_train_np = np.asarray(X_train)
X_train_np = X_train_np.reshape(np.shape(X_train)[0],170,1000,1)
np.shape(X_train_np)
X_test_np = np.asarray(X_test)
X_test_np = X_test_np.reshape(np.shape(X_test)[0],170,1000,1)

In [ ]:
model.fit(X_train_np, y_train_hot, batch_size=10, epochs=40, verbose=1, validation_data=(X_test_np, y_test_hot))

In [13]:
X2=[]
y2=[]
z2=[]

In [14]:
files = glob.glob("D:/iitd/SEM6/COP315/new_site_11/"+"**", recursive=False)
i=0
for file in files:
    mp3s = glob.glob(file+"/**.npy", recursive=False)
    i+=1
    print(file[32:])
    for mp3 in mp3s:
        a=np.load(mp3);
        if(np.shape(a)==(170,1000)):
            x=a-mean(a)
            X2.append(x[:,:])
            y2.append(i)
            z2.append(file[32:])

alexandrine_parakeet
bulbul
common_myna
drongo
hoopoe
house_crow
house_sparrow
indian_peafowl
indian_robin
jungle_babbler
koel
red_wattled_lapwing
rock_dove
tailorbird
treepie


In [15]:
np.shape(X2)

(476, 170, 1000)

In [16]:
X2_np = np.asarray(X2)
X2_np = X2_np.reshape(np.shape(X2_np)[0],170,1000,1)

In [17]:
y_pred=model.predict(X2_np)

In [18]:
np.shape(y_pred)

(476, 16)

In [19]:
yp=np.argmax(y_pred,axis=1)

In [20]:
print(accuracy_score(y2, yp))
from sklearn.metrics import classification_report
print(classification_report(y2, yp))

0.4789915966386555


D:\C\anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         8
           2       0.76      0.61      0.68        77
           3       0.34      0.39      0.37        76
           4       0.40      0.31      0.35        13
           5       0.09      0.22      0.12         9
           6       0.94      0.77      0.85        22
           7       0.00      0.00      0.00         0
           8       0.44      0.45      0.44        31
           9       1.00      0.37      0.54        46
          10       0.24      0.48      0.32        21
          11       0.82      0.81      0.82        52
          12       0.75      0.20      0.32        15
          13       0.47      0.52      0.49        44
          14       0.61      0.31      0.41        62
          15       0.00      0.00      0.00         0

   micro avg       0.48      0.48      0.48       476
   macro avg       0.46      0.36      0.38       476
weighted avg       0.61   

In [ ]:
model.save_weights("11_classes_xeno.hdf5")

In [56]:
X2=[]
file="D:/iitd/SEM6/COP315/test/"
mp3s = glob.glob(file+"/**.npy", recursive=False)
print(file[32:])
for mp3 in mp3s:
    a=np.load(mp3);
    if(np.shape(a)==(170,1000)):
        x=a-mean(a)
        X2.append(x[:,:])
n=np.shape(X2)[0]
X2_np = np.asarray(X2)
X2_np = X2_np.reshape(n,170,1000,1)

In [57]:
y_pred=model.predict(X2_np)
yp=np.argmax(y_pred,axis=1)

In [58]:
yp

array([14,  2,  4,  1,  1,  1, 11,  2, 11, 12,  9, 16, 11, 11, 11, 11, 11,
        2, 11, 11,  1,  1,  9,  9,  9, 15, 15, 15, 15,  2,  4,  2,  9,  9],
      dtype=int64)

In [37]:
class_dict={}
for i in range(np.shape(bird_df['class name'])[0]):
    class_dict[bird_df['class no'][i]]=bird_df['class name'][i][:]
print(class_dict)


{1: 'alexandrine_parakeet', 2: 'bulbul', 3: 'common_myna', 4: 'house_crow', 5: 'house_sparrow', 6: 'indian_peafowl', 7: 'indian_robin', 8: 'jungle_babbler', 9: 'koel', 10: 'rock_dove', 11: 'tailor_bird'}


In [38]:
import pickle

with open('D:/iitd/SEM6/COP315/bird_class_11_dict.pickle', 'wb') as handle:
    pickle.dump(class_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [67]:
for i in range(np.shape(yp)[0]):
    print(class_dict[yp[i]])

pec\rose-ringed_parakeet
pec\brahamini_myna
pec\common_myna
pec\alexandrine_parakeet
pec\alexandrine_parakeet
pec\alexandrine_parakeet
pec\red_vented_bulbul
pec\brahamini_myna
pec\red_vented_bulbul
pec\red_whiskered_bulbul
pec\jungle_babbler
pec\yellow_footed_pigeon
pec\red_vented_bulbul
pec\red_vented_bulbul
pec\red_vented_bulbul
pec\red_vented_bulbul
pec\red_vented_bulbul
pec\brahamini_myna
pec\red_vented_bulbul
pec\red_vented_bulbul
pec\alexandrine_parakeet
pec\alexandrine_parakeet
pec\jungle_babbler
pec\jungle_babbler
pec\jungle_babbler
pec\tailor_bird
pec\tailor_bird
pec\tailor_bird
pec\tailor_bird
pec\brahamini_myna
pec\common_myna
pec\brahamini_myna
pec\jungle_babbler
pec\jungle_babbler


In [69]:
model.save_weights('my_model.h5')